In [2]:
# -*- coding:utf-8 -*-
#
#       Navier-Stokes flow in a driven cavity
#
#       Emmanuel Dormy (2021)
#

import sys
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as lg


# affichage graphique
import matplotlib.pyplot as plt
plt.ion()


def BuildLAPD():
    """
    Laplacian with Dirichlet BC

    """
    # Dropping ghost points (-2)
    NXi = nx
    NYi = ny

    # 1D Laplace operator

    # X-axis
    # Diagonal terms
    dataNXi = [np.ones(NXi), -2 * np.ones(NXi), np.ones(NXi)]

    # Boundary conditions
    dataNXi[1][0] = -3.  # Dirichlet
    dataNXi[1][-1] = -3.  # Dirichlet

    # Y-axis
    # Diagonal terms
    dataNYi = [np.ones(NYi), -2 * np.ones(NYi), np.ones(NYi)]

    # Boundary conditions
    dataNYi[1][0] = -3.  # Dirichlet
    dataNYi[1][-1] = -3.  # Dirichlet

    # Their positions
    offsets = np.array([-1, 0, 1])
    DXX = sp.dia_matrix((dataNXi, offsets), shape=(NXi, NXi)) * dx_2
    DYY = sp.dia_matrix((dataNYi, offsets), shape=(NYi, NYi)) * dy_2
    # print(DXX.todense())
    # print(DYY.todense())

    # 2D Laplace operator
    LAPD = sp.kron(sp.eye(NYi, NYi), DXX) + sp.kron(DYY, sp.eye(NXi, NXi))

    return LAPD


def BuildLAPN():
    """
    Laplacian matrix for Phi with Neumann BC

    The value is set at one point (here [0][1]) to ensure uniqueness

    """
    # Dropping ghost points (-2)
    NXi = nx
    NYi = ny

    # 1D Laplace operator

    # X-axis
    # Diagonal terms
    dataNXi = [np.ones(NXi), -2 * np.ones(NXi), np.ones(NXi)]

    # Boundary conditions
    dataNXi[1][0] = -1.  # Neumann
    dataNXi[1][-1] = -1.  # Neumann

    # Y-axis
    # Diagonal terms
    dataNYi = [np.ones(NYi), -2 * np.ones(NYi), np.ones(NYi)]

    # Boundary conditions
    dataNYi[1][0] = -1.  # Neumann
    dataNYi[1][-1] = -1.  # Neumann

    # Their positions
    offsets = np.array([-1, 0, 1])
    DXX = sp.dia_matrix((dataNXi, offsets), shape=(NXi, NXi)) * dx_2
    DYY = sp.dia_matrix((dataNYi, offsets), shape=(NYi, NYi)) * dy_2
    # print(DXX.todense())
    # print(DYY.todense())

    # 2D Laplace operator
    LAP = sp.kron(DXX, sp.eye(NYi, NYi)) + sp.kron(sp.eye(NXi, NXi), DYY)

    # BUILD CORRECTION MATRIX

    # Upper Diagonal terms
    # dataNYNXi = [np.zeros(NYi * NXi)]
    # offset = np.array([1])

    # # Fix coef: 2+(-1) = 1 ==> Dirichlet at a single point
    # dataNYNXi[0][1] = -1 * dx_2

    # LAP0 = sp.dia_matrix((dataNYNXi, offset), shape=(NYi * NXi, NYi * NXi))
    dataNYNXi = [np.zeros(NYi * NXi), np.zeros(NYi * NXi)]
    offset = np.array([-1, 1])

    # Fix coef: 2+(-1) = 1 ==> Dirichlet at a single point
    dataNYNXi[0][0] = -1 * dx_2
    dataNYNXi[1][1] = -1 * dx_2

    LAP0 = sp.dia_matrix((dataNYNXi, offset), shape=(NYi * NXi, NYi * NXi))

    # tmp = LAP + LAP0
    # print(LAP.todense())
    # print(LAP0.todense())
    # print(tmp.todense())

    return LAP + LAP0


def LUdecomposition(LAP):
    """
    return the Incomplete LU decomposition
    of a sparse matrix LAP
    """
    return lg.splu(LAP.tocsc(),)


def Resolve(splu, RHS):
    """
    solve the system

    SPLU * x = RHS

    Args:
    --RHS: 2D array((NY,NX))
    --splu: (Incomplete) LU decomposed matrix
            shape (NY*NX, NY*NX)

    Return: x = array[NY,NX]

    Rem1: taille matrice fonction des CL

    """
    # array 2D -> array 1D
    f2 = RHS.ravel()

    # Solving the linear system
    x = splu.solve(f2)

    return x.reshape(RHS.shape)


####
def Laplacien(x):
    """
    calcule le laplacien scalaire
    du champ scalaire x(i,j)

    pas de termes de bord car ghost points

    """
    rst = np.empty((NX, NY))

    coef0 = -2 * (dx_2 + dy_2)

    rst[1:-1, 1:-1] = ((x[2:, 1:-1] + x[:-2, 1:-1]) * dx_2 +
                       (x[1:-1, 2:] + x[1:-1, :-2]) * dy_2 +
                       (x[1:-1, 1:-1]) * coef0)
    return rst


def divergence(u, v):
    """
    divergence avec points fantomes
    ne jamais utiliser les valeurs au bord

    """
    tmp = np.empty((NX, NY))

    tmp[1:-1, 1:-1] = (
        (u[2:, 1:-1] - u[:-2, 1:-1]) / dx / 2 +
        (v[1:-1, 2:] - v[1:-1, :-2]) / dy / 2)

    return tmp


###
def VelocityGhostPoints(u, v):
    # NO SLIP BC
    # bottom
    u[:, 0] = -u[:, 1]
    v[:, 0] = -v[:, 1]
    # top
    u[:, -1] = 2 - u[:, -2]
    v[:, -1] = -v[:, -2]
    # right
    u[0, :] = -u[1, :]
    v[0, :] = -v[1, :]
    # left
    u[-1, :] = -u[-2, :]
    v[-1, :] = -v[-2, :]


def PhiGhostPoints(phi):
    """
    copie les points fantomes
    tjrs Neumann

    global ==> pas de return

    """
    # bottom
    phi[:, 0] = phi[:, 1]
    # top
    phi[:, -1] = phi[:, -2]
    # right
    phi[0, :] = phi[1, :]
    # left
    phi[-1, :] = phi[-2, :]


####

####
def Semilag2(u, v, q):
    """
    Second order semi-Lagrangian advection
    """

    qstar = Semilag(u, v, q)
    qtilde = Semilag(-u, -v, qstar)
    qstar = q + (q - qtilde) / 2
    ADVq = Semilag(u, v, qstar)

    return ADVq

####


def Semilag(u, v, q):
    """
    1st order semi-Lagrangian advection
    """
    ADVq = np.zeros((NX, NY))

# Matrices where 1 is right, 0 is left or center
    Mx2 = np.sign(np.sign(u[1:-1, 1:-1]) + 1.)
    Mx1 = 1. - Mx2

# Matrices where 1 is up, 0 is down or center
    My2 = np.sign(np.sign(v[1:-1, 1:-1]) + 1.)
    My1 = 1. - My2

# Matrices of absolute values for u and v
    au = abs(u[1:-1, 1:-1])
    av = abs(v[1:-1, 1:-1])

# Matrices of coefficients respectively central, external, same x, same y
    Cc = (dx - au * dt) * (dy - av * dt) / dx / dy
    Ce = dt * dt * au * av / dx / dy
    Cmx = (dx - au * dt) * av * dt / dx / dy
    Cmy = dt * au * (dy - dt * av) / dx / dy


# Computes the advected quantity
    ADVq[1:-1, 1:-1] = (Cc * q[1:-1, 1:-1] +
                        Ce * (Mx1 * My1 * q[2:, 2:] +
                              Mx2 * My1 * q[:-2, 2:] +
                              Mx1 * My2 * q[2:, :-2] +
                              Mx2 * My2 * q[:-2, :-2]) +
                        Cmx * (My1 * q[1:-1, 2:] +
                               My2 * q[1:-1, :-2]) +
                        Cmy * (Mx1 * q[2:, 1:-1] +
                               Mx2 * q[:-2, 1:-1]))

    return ADVq

#########################################
# MAIN: Programme principal
#########################################


# Domain Size
# aspect_ratio = LY/LX

aspect_ratio = float(1.)
LY = float(1.)
LX = LY / aspect_ratio

# Grid Size

# (incuding ghost points)

NX = int(56)
NY = int(70)

# Number of iterations for the divergence
NI = 1


# Taille du domaine reel
nx = NX - 2
ny = NY - 2

# Nombre d'iterations
nitermax = int(10001)

# Modulo
modulo = int(1000)

# CONDITIONS INITIALES

# Valeurs initiales des vitesses
u = np.zeros((NX, NY))
v = np.zeros((NX, NY))

# Elements differentiels

dx = LX / (nx)
dy = LY / (ny)

dx_2 = 1. / (dx * dx)
dy_2 = 1. / (dy * dy)

# Grid, for plotting only
x = np.linspace(dx / 2, LX - dx / 2, nx)
y = np.linspace(dy / 2, LY - dy / 2, ny)
[xx, yy] = np.meshgrid(x, y)

# ATTENTION: dt_init calculer la CFL a chaque iteration...
dt = 0.0001

t = 0.  # total time

# parameters (Reynolds number)
Re = 100.0


# Tableaux avec points fantomes
# Matrices dans lesquelles se trouvent les extrapolations
ADVu = np.zeros((NX, NY))
ADVv = np.zeros((NX, NY))

# Definition des matrices ustar et vstar
ustar = np.zeros((NX, NY))
vstar = np.zeros((NX, NY))

# Definition de divstar
divstar = np.zeros((NX, NY))

# Definition de la pression phi
phi = np.zeros((NX, NY))
gradphix = np.zeros((NX, NY))
gradphiy = np.zeros((NX, NY))


# CONSTRUCTION des matrices et LU decomposition

# Matrix construction for projection step
LAPN = BuildLAPN()
LUPN = LUdecomposition(LAPN)


################
# MAIN LOOP
tStart = t
tEnd = 1
L_U = np.zeros((NX, NY))
# L_adve_U = np.zeros((2000,NX, NY))
# L_adve_V = np.zeros((2000,NX, NY))
# L_diff_U = np.zeros((2000,NX, NY))
# L_diff_V = np.zeros((2000,NX, NY))
# L_div = np.zeros((200,NX, NY))
# L_phi = np.zeros((2000,NX, NY))

iter = 0
while t < tEnd - 0.0000000001:

    t += dt

# for k in range(NI):

    # Advection semi-Lagrangienne
    ADVu = Semilag2(u, v, u)
    ADVv = Semilag2(u, v, v)

    # L_adve_U[iter] = ADVu
    # L_adve_V[iter] = ADVv

    # Diffusion step
    ustar = ADVu + dt * Laplacien(u) / Re
    vstar = ADVv + dt * Laplacien(v) / Re

    # Ghost points update
    VelocityGhostPoints(ustar, vstar)

    # L_diff_U[iter] = ustar
    # L_diff_V[iter] = vstar

    # Update divstar
    divstar = divergence(ustar, vstar)
    divstar = divstar - np.mean(divstar[1:-1, 1:-1])

    # L_div[iter] = divstar

    # Solving the linear system
    phi[1:-1, 1:-1] = Resolve(LUPN, RHS=divstar[1:-1, 1:-1])

    # update Pressure ghost points
    PhiGhostPoints(phi)

    # L_phi[iter] = phi

    # Update gradphi
    gradphix[1:-1, :] = (phi[2:, :] - phi[:-2, :]) / dx / 2
    gradphiy[:, 1:-1] = (phi[:, 2:] - phi[:, :-2]) / dy / 2

    # Project u
    u = ustar - gradphix
    v = vstar - gradphiy

    # Mise a jour des points fantomes
    # pour le champ de vitesse et T

    VelocityGhostPoints(u, v)

    print("t = ", t)
    # L_U = np.sqrt(u[1:-1, 1:-1]**2 + v[1:-1, 1:-1]**2)
    if (t > tEnd - 0.0000000001):
        L_U = u
        # FIGURE draw works only if plt.ion()
        # plotlabel = "t = %1.5f" % (t)
        # plt.clf()
        # plt.title(plotlabel)
        # plt.quiver(xx, yy, np.transpose(
        #     u[1:-1, 1:-1]), np.transpose(v[1:-1, 1:-1]), 4)
        # plt.axis('image')
        # plt.draw()
        # plt.pause(0.1)

t =  0.0001
t =  0.0002
t =  0.00030000000000000003
t =  0.0004
t =  0.0005
t =  0.0006000000000000001
t =  0.0007000000000000001
t =  0.0008000000000000001
t =  0.0009000000000000002
t =  0.0010000000000000002
t =  0.0011000000000000003
t =  0.0012000000000000003
t =  0.0013000000000000004
t =  0.0014000000000000004
t =  0.0015000000000000005
t =  0.0016000000000000005
t =  0.0017000000000000006
t =  0.0018000000000000006
t =  0.0019000000000000006
t =  0.0020000000000000005
t =  0.0021000000000000003
t =  0.0022
t =  0.0023
t =  0.0024
t =  0.0024999999999999996
t =  0.0025999999999999994
t =  0.0026999999999999993
t =  0.002799999999999999
t =  0.002899999999999999
t =  0.0029999999999999988
t =  0.0030999999999999986
t =  0.0031999999999999984
t =  0.0032999999999999982
t =  0.003399999999999998
t =  0.003499999999999998
t =  0.0035999999999999977
t =  0.0036999999999999976
t =  0.0037999999999999974
t =  0.0038999999999999972
t =  0.0039999999999999975
t =  0.004099999999999998
t 

In [4]:
np.set_printoptions(threshold=np.inf, linewidth=1800)

In [5]:
L_U

array([[ 1.79607661e-05, -1.79607661e-05, -2.04261255e-05, -1.53848817e-05, -6.14938972e-06,  1.46333199e-06,  8.33515416e-06,  1.32573665e-05,  1.72548808e-05,  2.00596808e-05,  2.23069162e-05,  2.39161433e-05,  2.52405263e-05,  2.62477670e-05,  2.71341567e-05,  2.78866690e-05,  2.86171822e-05,  2.93218201e-05,  3.00656574e-05,  3.08487213e-05,  3.17092305e-05,  3.26485878e-05,  3.36894986e-05,  3.48340266e-05,  3.60963527e-05,  3.74796019e-05,  3.89940967e-05,  4.06451126e-05,  4.24425537e-05,  4.43954961e-05,  4.65163767e-05,  4.88204330e-05,  5.13257245e-05,  5.40571160e-05,  5.70421806e-05,  6.03206188e-05,  6.39350471e-05,  6.79478747e-05,  7.24252453e-05,  7.74641245e-05,  8.31681611e-05,  8.96865677e-05,  9.71857326e-05,  1.05893206e-04,  1.16088991e-04,  1.28116329e-04,  1.42483364e-04,  1.59698271e-04,  1.80781228e-04,  2.06455878e-04,  2.39020741e-04,  2.79423344e-04,  3.33489342e-04,  4.02255198e-04,  5.02252029e-04,  6.34231296e-04,  8.50103599e-04,  1.14983159e-03,  1.709

In [14]:
for X in range(nx):
  for Y in range(ny):
    if abs(L_divstar[X,Y])<1.e-8:
      L_divstar[X,Y]= int(0)

In [23]:
print(L_u)

[[ 0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -2.]
 [-0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [-0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [-0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

In [5]:
nx = 8
ny = 5

NX +=2
NY += 2
dx = 0.01
dy = 0.01
dx_2 = 1/(dx**2)
dy_2 = 1/(dy**2)

# Matrix construction for projection step
LAPN = BuildLAPN()
LUPN = LUdecomposition(LAPN)

In [ ]:
L_


In [8]:
LAPN.inverse()

AttributeError: 'csr_matrix' object has no attribute 'inverse'

In [106]:
print(LAPN.todense())

[[-20000.  10000.      0.      0.      0.  10000.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.]
 [ 10000. -30000.  10000.      0.      0.      0.  10000.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.]
 [     0.  10000. -30000.  10000.      0.      0.      0.  10000.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.]
 [     0.      0.  10000. -3000

In [107]:
RHS = np.arange(nx*ny)
RHS=RHS.reshape((ny,nx))

In [108]:
RHS.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])

In [109]:
x = Resolve(LUPN, RHS=RHS)

In [100]:
x

array([[-0.04044188, -0.078     , -0.09853107, -0.10995128, -0.11517643, -0.08088376, -0.09492705, -0.10744191],
       [-0.11584635, -0.12000158, -0.10678236, -0.11278253, -0.11976318, -0.12519064, -0.12808195, -0.12568078],
       [-0.12855753, -0.13243765, -0.13577108, -0.13765362, -0.14020246, -0.14172915, -0.14395881, -0.1460024 ],
       [-0.14720785, -0.15119744, -0.15209779, -0.15346605, -0.15477188, -0.15556751, -0.15879208, -0.15939852],
       [-0.1603357 , -0.16125156, -0.1618228 , -0.16278029, -0.1632685 , -0.16402669, -0.16477586, -0.16524933]])

In [110]:
x

array([[-8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13],
       [-8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13],
       [-8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13],
       [-8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13],
       [-8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13, -8.5761907e+13]])